### torchtext 라이브러리로 텍스트 분류 <hr>
- 1단계 : 데이터 전처리 => 숫자형식으로 변환
- 2단계 : 모델 구현

[1-1] 데이터 준비 => 내장 데이터셋 활용
- AGNEWS 데이터셋 반복자 : 레이블(label) + 문장의 튜풀(tuple) 형태

In [34]:
# !pip install torchdata

In [35]:
import torchtext
torchtext.__version__

'0.17.2'

In [36]:
import torch
from torchtext.datasets import AG_NEWS

# DataPipe 타입 >>> iterator 타입 형변환
train_iter = iter(AG_NEWS(split = 'train'))

In [37]:
# 데이터 확인 => (label, text), label : 1 ~ 4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

[2] 데이터 전처리 파이프라인 준비
- 어휘집(vocab), 단어 벡터(word vector), 토크나이저(tokenizer)
- 가공되지 않은 텍스트 문자열에 대한 데이터 처리 빌딩 블록
- 일반적인 NLP 데이터 처리
    - 첫번째 단계 : 가공되지 않은 학습 데이터셋으로 어휘집 생성  
    ==> 토큰 목록 또는 반복자 받는 내장 팩토리 함수 (factory function) : ``bulid_vocab_from_iterator``
    - 사용자는 어휘집에 추가할 특수 기호(special symbol) 전달 가능


In [38]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [39]:
# 여기에 적기

In [40]:
# 토커나이즈 생성
tokenizer = get_tokenizer('basic_english')

# 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split = 'train')

In [41]:
# 토큰 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트 => 텍스트 토큰화
        yield tokenizer(text)

#### build_vocab_from_iterator의 파라미터
- iterator – Iterator used to build Vocab. Must yield list or iterator of tokens. 어휘사전을 만들 때 쓰이는 iterator.
- min_freq – The minimum frequency needed to include a token in the vocabulary. 최소 빈도 수 (최소 빈도 수를 늘리면 어휘사전에 들어가는 숫자 감소)
- specials – Special symbols to add. The order of supplied tokens will be preserved.
- special_first – Indicates whether to insert symbols at the beginning or at the end.
- max_tokens – If provided, creates the vocab from the max_tokens - len(specials) most frequent tokens.

In [42]:
# 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials = ['<unk>'])

# <UNK> 인덱스 0으로 설정
vocab.set_default_index(vocab['<unk>'])

In [43]:
vocab(['<unk>', 'here', 'is', 'an', 'example'])

[0, 475, 21, 30, 5297]

In [44]:
# 텍스트 => 정수 인코딩
text_pipeline = lambda x : vocab(tokenizer(x))

# 레이블 => 정수 인코딩
label_pipeline = lambda x : int(x) - 1

[3] 데이터 배치(batch)와 반복자 생성
- torch.utils.data.DataLoader : getitem(), len() 구현한 맵 형태(map-style)
- collate_fn() : DataLoader로부터 생성된 샘플 배치 함수  
==> 입력 : DataLoader에 배치 크기(batch size)가 있는 배치(batch) 데이터

In [45]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 배치 크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]

    # 1개씩 뉴스기사, 라벨 추출해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype = torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))
    
    label_list = torch.tensor(label_list, dtype = torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim = 0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

In [46]:
train_iter = AG_NEWS(split = 'train')
dataloader = DataLoader(train_iter,
                        batch_size = 8,
                        shuffle = True,
                        collate_fn = collate_batch)

In [47]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f'num class : {num_class}   vocab size : {vocab_size}')

num class : 4   vocab size : 95811


In [48]:
for labels, texts, offsets in dataloader:
    print(labels, texts, offsets)
    break

tensor([3, 2, 3, 0, 3, 0, 1, 0]) tensor([  510,  8920,   761,  4363,  3286,    13,    31,    14,    31,    15,
          510,  3612,    22,    47,  3286,    11,   761,  4363,    56,     3,
         2758,   211,   420,    10,     2,   466,  1444,     6,  2198,   709,
          207,     3,   103,    21,  9551,    24,     2,   128,   790,     8,
          103,   306,   453,  1706,   104,   170,     4,    23,  8347,    11,
            5,  1413,     6,  7359,     1,  4130,    12,     9,  4320,   400,
         2294,   354,   588,     3, 49973,    84,  4320,   278,   400,    11,
         4130,  2196,     3,   358,    12,     9,   238,  5521,     3,  3336,
            2,    54,    12,     9,   373,    11,     5,   395,     1,   108,
          106,   354,   588,     8,    33,   465,     4,   376,    96,    34,
          664,    10,   503,    56,     3, 49973,  1352,    26,     1, 13282,
            7, 15915,  8046,   200,   101, 44329,    13, 13282, 44329,    14,
         7551,   801, 15915,  1

[4] 모델 설계

In [49]:
from torch import nn

# 모델 설계
# 입력층 : EmbeddingBag Layer - 레이어와 분류(classification) 목적을 위한 선형 레이어, 텍스트의 길이는 오프셋(offset)
# 은닉층 : Linear - 4개 클래스 분류
class TextModel(nn.Module):
    def __init__(self, VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS):
        super().__init__()
        # 모델 구성 층 정의
        self.embedding = nn.EmbeddingBag(VOCAB_SIZE, EMBEDD_DIM, sparse = False)
        # self.rnn == nn.RNN(EMBEDD_DIM, HIDDEN_DIM)
        self.fc = nn.Linear(EMBEDD_DIM, NUM_CLASS)
        self.init_weights()

    # 가중치 초기화
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
    # 순방향 학습 진행
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        # output, _ = self.rnn(embedded)
        return self.fc(embedded)

In [50]:
# 학습관련 하이퍼파라미터와 인스턴스
HIDDEN_SIZE = 3
EMBEDD_DIM = 64
VOCAB_SIZE = len(vocab)
NUM_CLASS = len(set([label for label, _ in train_iter]))
EPOCHS = 10
LR = 5
BATCH_SIZE = 64

c:\ProgramData\anaconda3\envs\Torch_NLP38\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [51]:
from torch import optim

# 학습관련 인스턴스
MODEL = TextModel(VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS).to(device)

CRITERION = nn.CrossEntropyLoss()
OPTIMIZER = optim.SGD(MODEL.parameters(), lr = LR)
SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER, step_size = 1.0, gamma = 0.1)  # learning rate를 줄이는 용도 

In [52]:
def train(model, dataloader, optimizer, criterion, epoch):
    
    model.train()

    # 학습 평가 관련 변수들
    total_acc, total_count = 0, 0
    log_interval = 300

    # 배치 학습 진행
    for idx, (label, text, offsets) in enumerate(dataloader):

        label, text, offsets = label.to(device), text.to(device), offsets.to(device)

        # 학습진행
        predicted_label = MODEL(text, offsets)
        # print(f'predicted_label : {predicted_label.shape}  label : {label.shape}')

        # 손실 계산 및 W, b 업데이트
        OPTIMIZER.zero_grad()
        loss = CRITERION(predicted_label, label)

        # 기울기 소실 및 폭주 예방을 위한 양극단 값 자르기
        torch.nn.utils.clip_grad_norm_(MODEL.parameters(), max_norm = 0.1)
        OPTIMIZER.step()

        # 배치 학습 평가
        total_acc += (predicted_label.argmax(1) ==label).sum().item()
        total_count += label.size(0)

        if idx % log_interval == 0 and idx > 0:
            print(f"epoch : {epoch} batch : {idx} loss : {loss.item()}")
            print(f"Accuracy : {total_acc/total_count}")
            total_acc, total_count = 0,0
            
            break

In [53]:
def evaluate(model, dataloader, criterion):
    
    model.eval()    
    total_acc, total_count = 0, 0
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = CRITERION(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            
    return total_acc / total_count

In [54]:
def predict(model, text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text), dtype=torch.int64).to(device)
        text = text.unsqueeze(0)
        offsets = torch.tensor([0]).to(device)
        predicted_label = model(text, offsets)
        return predicted_label.argmax(1).item() + 1

In [55]:
# 학습 진행
for epoch in range(1, EPOCHS+1):
    train(MODEL, dataloader, OPTIMIZER, CRITERION, epoch)
    accu_val = evaluate(MODEL, dataloader, CRITERION)
    print(f"epoch : {epoch} Accuracy : {accu_val}")
    SCHEDULER.step()

epoch : 1 batch : 300 loss : 1.4147255420684814
Accuracy : 0.24543189368770765
epoch : 1 Accuracy : 0.25021666666666664
epoch : 2 batch : 300 loss : 1.4797996282577515
Accuracy : 0.24335548172757476
epoch : 2 Accuracy : 0.25021666666666664
epoch : 3 batch : 300 loss : 1.4044699668884277
Accuracy : 0.2558139534883721
epoch : 3 Accuracy : 0.25021666666666664
epoch : 4 batch : 300 loss : 1.3760309219360352
Accuracy : 0.2479235880398671
epoch : 4 Accuracy : 0.25021666666666664
epoch : 5 batch : 300 loss : 1.3967318534851074
Accuracy : 0.2404485049833887
epoch : 5 Accuracy : 0.25021666666666664
epoch : 6 batch : 300 loss : 1.3563001155853271
Accuracy : 0.23421926910299004
epoch : 6 Accuracy : 0.25021666666666664
epoch : 7 batch : 300 loss : 1.3982785940170288
Accuracy : 0.24750830564784054
epoch : 7 Accuracy : 0.25021666666666664
epoch : 8 batch : 300 loss : 1.3803147077560425
Accuracy : 0.2520764119601329
epoch : 8 Accuracy : 0.25021666666666664
epoch : 9 batch : 300 loss : 1.3584797382354